In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from scipy.optimize import minimize

In [2]:
data = fpa_data = pd.read_stata('chile.dta')

In [3]:
olsdata = data.copy()
olsdata[['routput','totlab','rcapstock']] = np.log(olsdata[['routput','totlab','rcapstock']])

olsmodel = smf.ols('routput ~totlab + rcapstock',olsdata)
olsresult = olsmodel.fit()
olsresult.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                routput   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                 3.452e+04
Date:                Wed, 08 May 2019   Prob (F-statistic):               0.00
Time:                        14:28:59   Log-Likelihood:                -23690.
No. Observations:               20730   AIC:                         4.739e+04
Df Residuals:                   20727   BIC:                         4.741e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.1804      0.024    214.199      0.000       5.133       5.228
totlab         0.8194      0.008    102.168      0.000       0.804       0.835
rcapstock      0.3317      0.004     91.134      0.000       0.325       0.339
==============================================================================
Omnibus:                     1840.946   Durbin-Watson:                   0.531
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3645.736
Skew:                           0.597   Prob(JB):                         0.00
Kurtosis:                       4.672   Cond. No.                         47.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
def add_fe(df, dfvars,groups):
    #apply the within transformation
    for var in dfvars: #,
        df[var+'fe'] = df[var]
        for fe in groups:
            dum = pd.get_dummies(fedata[fe])
            means = np.tile(fedata[var],len(dum.columns)).reshape(len(dum.columns),fedata.shape[0])
            means = np.array(means.transpose()*dum)
            means = means.sum(axis=0)/dum.sum(axis=0)
            means = np.tile(means,fedata.shape[0]).reshape(fedata.shape[0],len(dum.columns))
            means = (means*dum).sum(axis=1)
            df[var+'fe'] = df[var+'fe'] - means
    return df

fedata = olsdata.copy()
fedata = add_fe(fedata, ['routput','totlab','rcapstock'], ['id'])

In [6]:
femodel = smf.ols('routputfe ~totlabfe + rcapstockfe -1',fedata)
feresult = femodel.fit()
feresult.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              routputfe   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.307
Method:                 Least Squares   F-statistic:                     4592.
Date:                Wed, 08 May 2019   Prob (F-statistic):               0.00
Time:                        14:29:13   Log-Likelihood:                -6980.9
No. Observations:               20730   AIC:                         1.397e+04
Df Residuals:                   20728   BIC:                         1.398e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
totlabfe        0.5661      0.007     81.705      0.000       0.552       0.580
rcapstockfe     0.1374      0.004     30.618      0.000       0.129       0.146
==============================================================================
Omnibus:                     4357.853   Durbin-Watson:                   1.341
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            50938.914
Skew:                          -0.676   Prob(JB):                         0.00
Kurtosis:                      10.560   Cond. No.                         1.64
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""